<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Aperçu-des-données-nutritionnelles" data-toc-modified-id="Aperçu-des-données-nutritionnelles-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Aperçu des données nutritionnelles</a></span><ul class="toc-item"><li><span><a href="#Découvrir-des-exemples-de-produits-par-nutriscore" data-toc-modified-id="Découvrir-des-exemples-de-produits-par-nutriscore-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Découvrir des exemples de produits par nutriscore</a></span></li></ul></li></ul></div>

In [57]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, FloatSlider, Button, Label
import ipywidgets as widgets
from ipywidgets import Layout, HBox, VBox


from IPython.display import Image, HTML
from IPython.display import display as disp
from IPython import display

from IPython.display import Image, HTML
from IPython.display import display as disp

#from zipfile import ZipFile, Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import voila
import streamlit as st
import time
import threading

#import statsmodels.api as sm
import plotly.graph_objects as go
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
#from functions import *

import warnings
warnings.filterwarnings('ignore')

# Show all code cells outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'



import ipywidgets as widgets
from ipywidgets import interact, interact_manual
#interact(f, x=10)

In [74]:
# Graphic Interface

style = {'description_width': 'initial'}

limit_data2see = widgets.IntSlider(
    value=5,
    min=2,
    max=200,
    step=1,
    description='Nombre de produits:',
    disabled=False,
    style=style)






In [76]:
data_load = pd.read_csv("./data/PGclean_data.csv",low_memory=False) #index_col="product_name"
print("Des données optimisées d'OpenFoodFacts sont chargées avec succes")
#data_load.head(10)

Des données optimisées d'OpenFoodFacts sont chargées avec succes


In [77]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
# fig.add_trace(go.Bar(y=[1, 4, 3, 2]))
# fig.update_layout(title = 'Hello Figure')
# fig.show()

# Aperçu des données nutritionnelles #


Visualisez le type de données nettoyées:

In [78]:
data_load.head(5)

,Unnamed: 0,fat_100g,sugars_100g,fiber_100g,proteins_100g,energy_100g,nutriscore_score,nutriscore_grade,product_name
0,1433286,0.0,14.0,0.20,0.5,259.0,9.0,d,Jus de sureau
1,983645,15.0,33.2,0.90,4.1,1565.0,14.0,d,"8 Mini muffins, cœur fondant au fruits rouges"
2,667554,56.0,0.9,0.10,7.5,2276.0,21.0,e,Foie gras de canard entier du Sud-Ouest
3,787015,29.1,0.6,0.44,13.5,1372.0,21.0,e,Terrine de campagne pur porc
4,954612,30.0,0.0,0.00,32.0,1682.0,17.0,d,Copeaux Parmigiano Reggiano


## Découvrir des exemples de produits par nutriscore ##

Sélectionnez le nutriscore (entre A et E), pour voir une liste de produits

In [79]:
style = {'description_width': 'initial'}
limit_case = widgets.IntSlider(
    value=1000,
    min=100,
    max=5000,
    step=1,
    description='Max Number of Case:',
    disabled=False,
    style=style)

def update_df_length(limit_case,nutriscore):  #int #pd.dataframe
    #data_load = pd.read_csv("./data/PGclean_data.csv",low_memory=False)
    if nutriscore == 'tous':
        disp(data_load.head(limit_case)) #limit
    else:
        disp(data_load[data_load['nutriscore_grade']==nutriscore].head(limit_case))
    
    print('Nombre de produits affichés: {}  nutriscore: {}'.format(limit_case,nutriscore))
    
interact(update_df_length, limit_case=10,nutriscore=['tous','a','b','c','d','e'])

interactive(children=(IntSlider(value=10, description='limit_case', max=30, min=-10), Dropdown(description='nu…

<function __main__.update_df_length(limit_case, nutriscore)>